In [ ]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import print_hex, Str

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 4: Printing Department ---</h2><p>You ride the escalator down to the printing department. They're clearly getting ready for Christmas; they have lots of large rolls of paper everywhere, and there's even a massive printer in the corner (to handle the really <span title="This joke is stupid and I love it.">big</span> print jobs).</p>
<p>Decorating here will be easy: they can make their own decorations. What you really need is a way to get further into the North Pole base while the elevators are offline.</p>
<p>"Actually, maybe we can help with that," one of the Elves replies when you ask for help. "We're pretty sure there's a cafeteria on the other side of the back wall. If we could break through the wall, you'd be able to keep moving. It's too bad all of our forklifts are so busy moving those big rolls of paper around."</p>
<p>If you can optimize the work the forklifts are doing, maybe they would have time to spare to break through the wall.</p>
<p>The rolls of paper (<code>@</code>) are arranged on a large grid; the Elves even have a helpful diagram (your puzzle input) indicating where everything is located.</p>
<p>For example:</p>
<pre><code>..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.
</code></pre>
<p>The forklifts can only access a roll of paper if there are <em>fewer than four rolls of paper</em> in the eight adjacent positions. If you can figure out which rolls of paper the forklifts can access, they'll spend less time looking and more time breaking down the wall to the cafeteria.</p>
<p>In this example, there are <code><em>13</em></code> rolls of paper that can be accessed by a forklift (marked with <code>x</code>):</p>
<pre><code>..xx.xx@x.
x@@.@.@.@@
@@@@@.x.@@
@.@@@@..@.
x@.@@@@.@x
.@@@@@@@.@
.@.@.@.@@@
x.@@@.@@@@
.@@@@@@@@.
x.x.@@@.x.
</code></pre>
<p>Consider your complete diagram of the paper roll locations. <em>How many rolls of paper can be accessed by a forklift?</em></p>
</article>


In [ ]:
from collections.abc import Iterator


tests = [
    {
        "name": "Example",
        "s": """
            ..@@.@@@@.
            @@@.@.@.@@
            @@@@@.@.@@
            @.@@@@..@.
            @@.@@@@.@@
            .@@@@@@@.@
            .@.@.@.@@@
            @.@@@.@@@@
            .@@@@@@@@.
            @.@.@@@.@.
        """,
        "expected": 13,
    },
]


class Grid(Str):
    DIRECTIONS = (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)

    def __init__(self, s: str) -> None:
        self.grid = [list(l.strip()) for l in s.strip().splitlines()]

    def rolls_can_be_forklifted(self) -> int:
        return sum(1 for _ in self.forkliftable_rolls())

    def forkliftable_rolls(self) -> Iterator[tuple[int, int]]:
        rows, cols = len(self.grid), len(self.grid[0])
        return (
            (row, col)
            for row, col in product(range(rows), range(cols))
            if self.is_forkliftable_roll(row, col)
        )

    def is_forkliftable_roll(self, row, col) -> bool:
        return self.grid[row][col] == "@" and self.less_then_4_neigbor_rolls(row, col)

    def less_then_4_neigbor_rolls(self, row, col) -> bool:
        som = sum(1 for rn, cn in self.neighbors(row, col) if self.grid[rn][cn] == "@")
        return som < 4

    def neighbors(self, row: int, col: int) -> Iterator[tuple[int, int]]:
        rows, cols = len(self.grid), len(self.grid[0])

        return (
            (row + dr, col + dc)
            for dr, dc in self.DIRECTIONS
            if 0 <= row + dr < rows and 0 <= col + dc < cols
        )

    def __str__(self) -> str:
        return "\n".join("".join(l) for l in self.grid)


@test(tests=tests)
def part_I(s: str) -> int:
    g = Grid(s)
    return g.rolls_can_be_forklifted()

In [ ]:
with open("../input/day4.txt") as f:
    puzzle = f.read()

print(f"Part I: {part_I(puzzle)}")

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1445</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now, the Elves just need help accessing as much of the paper as they can.</p>
<p>Once a roll of paper can be accessed by a forklift, it can be <em>removed</em>. Once a roll of paper is removed, the forklifts might be able to access <em>more</em> rolls of paper, which they might also be able to remove. How many total rolls of paper could the Elves remove if they keep repeating this process?</p>
<p>Starting with the same example as above, here is one way you could remove as many rolls of paper as possible, using highlighted <code><em>@</em></code> to indicate that a roll of paper is about to be removed, and using <code>x</code> to indicate that a roll of paper was just removed:</p>
<pre><code>Initial state:
..<em>@</em><em>@</em>.<em>@</em><em>@</em>@<em>@</em>.
<em>@</em>@@.@.@.@@
@@@@@.<em>@</em>.@@
@.@@@@..@.
<em>@</em>@.@@@@.@<em>@</em>
.@@@@@@@.@
.@.@.@.@@@
<em>@</em>.@@@.@@@@
.@@@@@@@@.
<em>@</em>.<em>@</em>.@@@.<em>@</em>.

Remove 13 rolls of paper:
..xx.xx<em>@</em>x.
x@@.<em>@</em>.<em>@</em>.@<em>@</em>
<em>@</em>@@@@.x.@@
<em>@</em>.@@@@..<em>@</em>.
x@.@@@@.<em>@</em>x
.<em>@</em>@@@@@@.<em>@</em>
.<em>@</em>.@.@.@@@
x.@@@.@@@@
.<em>@</em>@@@@@@@.
x.x.@@@.x.

Remove 12 rolls of paper:
.......x..
.<em>@</em>@.x.x.<em>@</em>x
x@@@@...<em>@</em><em>@</em>
x.@@@@..x.
.<em>@</em>.@@@@.x.
.x@@@@@@.x
.x.@.@.@@<em>@</em>
..@@@.@@@@
.x<em>@</em>@@@@@@.
....@@@...

Remove 7 rolls of paper:
..........
.x<em>@</em>.....x.
.<em>@</em>@@@...xx
..@@@@....
.x.@@@@...
..<em>@</em>@@@@@..
...@.@.@@x
..<em>@</em>@@.@@@<em>@</em>
..x@@@@@@.
....@@@...

Remove 5 rolls of paper:
..........
..x.......
.x<em>@</em>@@.....
..@@@@....
...@@@@...
..x@@@@@..
...@.@.@@.
..x@@.@@@x
...@@@@@<em>@</em>.
....@@@...

Remove 2 rolls of paper:
..........
..........
..x@@.....
..<em>@</em>@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@x.
....@@@...

Remove 1 roll of paper:
..........
..........
...<em>@</em>@.....
..x@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
...x<em>@</em>.....
...@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
....x.....
...<em>@</em>@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
..........
...x@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...
</code></pre>

<p>Stop once no more rolls of paper are accessible by a forklift. In this example, a total of <code><em>43</em></code> rolls of paper can be removed.</p>
<p>Start with your original diagram. <em>How many rolls of paper in total can be removed by the Elves and their forklifts?</em></p>
</article>


In [54]:
tests = [
    {
        "name": "Example",
        "s": """
            ..@@.@@@@.
            @@@.@.@.@@
            @@@@@.@.@@
            @.@@@@..@.
            @@.@@@@.@@
            .@@@@@@@.@
            .@.@.@.@@@
            @.@@@.@@@@
            .@@@@@@@@.
            @.@.@@@.@.
        """,
        "expected": 43,
    },
]


class GridII(Grid):
    def rolls_can_be_forklifted(self, debug=False) -> int:
        count = 0
        rolls = list(self.forkliftable_rolls())

        if debug:
            print("Initial state:")
            print(self)
            print()

        while rolls:
            if debug:
                print(f"Remove {len(rolls)} rolls of paper:")

            for row, col in rolls:
                self.grid[row][col] = "."

            if debug:
                print(self)
                print()

            count += len(rolls)
            rolls = list(self.forkliftable_rolls())

        return count


@test(tests=tests)
def part_II(s: str) -> int:
    g = GridII(s)
    return g.rolls_can_be_forklifted()


Test Example passed, for part_II.
Success


<link href="style.css" rel="stylesheet"></link>


In [55]:
print(f"Part II: {part_II(puzzle)}")

Part II: 8317


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>8317</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
